Want to determine if the reason for the entanglement is because of 'clusters' forming in the graph. We will start small, and work from there.

In [2]:
using Pkg
Pkg.add(["LinearAlgebra", "ITensors", "ITensorMPS", "Statistics", "JLD2", "FileIO"])

using LinearAlgebra
using ITensors
using ITensorMPS
using Statistics
using JLD2, FileIO

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/.julia/environments/v1.11/Project.toml`
  [37e2e46d] + LinearAlgebra v1.11.0
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`


In [3]:
J = Δ = -1
# N_vals = 10:10:80
N_vals = [4]
σ_vals = [0.000, 0.001, 0.002]
μ = 1.0

NUM_SWEEPS = 30
MAX_BOND_DIM = 1000
ACC = 1e-10

AVG_OVER_PLOTS = 1

1

In [ ]:
function generate_fully_connected_wam(N::Int, σ::Float64; μ::Float64=1.0)
    """
    Create a weighted adjacency matrix for a fully connected graph of N nodes. μ should always be one really.
    """
 
    A = zeros(Float64, N, N)
    for i in 1:N
        for j in (i+1):N
            weight = μ + σ * randn() # weight from normal distribution with mean μ and std σ
            A[i, j] = weight
            A[j, i] = weight 
        end
    end
    return A
end 

function create_xxz_hamiltonian_mpo(N, adj_mat, J, Δ, sites)
    """Create the XXZ Hamiltonian as an MPO given an adjacency matrix."""
    ampo = OpSum()
    for i = 1:N-1
        for j = i+1:N
            weight = adj_mat[i, j]
            if weight != 0.0
                # XX and YY terms: S+S- + S-S+ = 2(SxSx + SySy)
                # So to get J(SxSx + SySy), we need J/2 * (S+S- + S-S+)
                ampo += weight * J/2, "S+", i, "S-", j
                ampo += weight * J/2, "S-", i, "S+", j
                # ZZ term
                ampo += weight * J * Δ, "Sz", i, "Sz", j
            end
        end
    end
    H = MPO(ampo, sites)
    return H
end 

function solve_xxz_hamiltonian_dmrg(H, ψ0, sweeps::Int=10, bond_dim::Int=1000, cutoff::Float64=1E-14)
    """Solves the XXZ Hamiltonian using DMRG with given parameters. Returns the ground state energy and MPS. """
    swps = Sweeps(sweeps)
    setmaxdim!(swps, bond_dim)
    setcutoff!(swps, cutoff)
    E, ψ = dmrg(H, ψ0, swps; outputlevel=0)
    return E, ψ # only ground state and ground state wavefunction
end

function create_MPS(L::Int, conserve_qns::Bool=true)
    """Create a random MPS for a spin-1/2 chain of length L with bond dimension Χ."""
    # create a site set for a spin-1/2 chain
    sites = siteinds("S=1/2", L; conserve_qns=conserve_qns) # conserve total Sz

    # create a random MPS with bond dimension Χ
    init_state = [isodd(i) ? "Up" : "Dn" for i = 1:L] # antiferromagnetic ground state
    # THIS IS IMPORTANT - SEE NOTE BELOW
    # it sets the subspace of states we are allowed to explore
    # for example, this init_state means we only explore states with total Sz = 0 (i.e. zero magnetisation)
    # this is a reasonable assumption for positive J, but not for negative J
    # if we want to explore ferromagnetic states (negative J), we would need a different init_state
    # USE create_custom_MPS TO SET A DIFFERENT INIT STATE
    ψ0 = MPS(sites, init_state)
    return ψ0, sites
end

function find_ground_state_mps(N, J, Δ, σ, μ)
    ψ_mps_N, sites_N = create_MPS(N)
    wam_N = generate_fully_connected_wam(N, σ; μ)
    H_N = create_xxz_hamiltonian_mpo(N, wam_N, J, Δ, sites_N)
    _, ψ_gs_N = solve_xxz_hamiltonian_dmrg(H_N, ψ_mps_N, NUM_SWEEPS, MAX_BOND_DIM, ACC)
    return ψ_gs_N, wam_N
end

function get_entanglement_data(psi, b)
    orthogonalize!(psi, b)
    s = siteind(psi, b)
    
    local U, S, V
    if b==1
        # no link to left, edge case
        U, S, V = svd(psi[b], (s,))
    else
        l = linkind(psi, b-1)
        U, S, V = svd(psi[b], (l, s))
    end

    schmidt_coefs = diag(S)
    schmidt_spectrum = schmidt_coefs .^ 2 # square all schmidt_coefs
    schmidt_spectrum = schmidt_spectrum[schmidt_spectrum .> 1e-15] # filter out really small p so we don't get NaN entropies
    schmidt_spectrum ./= sum(schmidt_spectrum) # normalise

    vn_entropy = -sum(p * log(p) for p in schmidt_spectrum) # von neuman entropy

    return schmidt_coefs, vn_entropy
end

get_entanglement_data (generic function with 1 method)